<a href="https://colab.research.google.com/github/NREL/boptest-service/blob/documentation_readme_changes/docs/Introduction_to_BOPTEST_Service_APIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BOPTEST Service APIs

This is an introduction to the BOPTEST-Service APIs. Using BOPTEST version 0.3. These APIS are based on HTTP REST framework. 


## Authentication
We will soon begin optional authentication with the BOPTEST dashboard username. You can create a BOPTEST username in the dashboard.boptest.net website. This username will be sent along with the API calls in as the Authentication parameter in the API header. If you are authenticated, you will be able to track your test status on the dashboard. 



We start with a short setup that imports the `requests` and `json` Python modules. We also define the url to the BOPTEST server. We have set it to `https://api.boptest.net` as default but feel free to change it to another server if you want to. 

In [ ]:
import requests
import json

BOPTEST_URL = 'https://api.boptest.net'

def return_pretty_JSON(json_object):
  return json.dumps(json_object, indent=2)

def print_BOPTEST_response(response):
  print(response['message'])

  if response['status'] == 200:
    payload = return_pretty_JSON(response['payload'])
    print(f'The payload is {payload}')


## What are the test cases that I can use? : The `/testcases` API

The `/testcases` API returns the testcases that are already available for use on the BOPTEST server. This is an GET request which is prefixed by the BOPTEST server URL. 

In [ ]:
all_testcases = requests.get(f"{BOPTEST_URL}/testcases")
print(return_pretty_JSON(all_testcases.json()[:8])) # Printing out the first eight testcases

[
  {
    "testcaseid": "bestest_air"
  },
  {
    "testcaseid": "bestest_hydronic"
  },
  {
    "testcaseid": "bestest_hydronic_heat_pump"
  },
  {
    "testcaseid": "multizone_residential_hydronic"
  },
  {
    "testcaseid": "singlezone_commercial_hydronic"
  },
  {
    "testcaseid": "testcase1"
  },
  {
    "testcaseid": "testcase2"
  },
  {
    "testcaseid": "testcase3"
  }
]


## Is a specific test case available? The GET `testcases/{testcase_name}` API

If you want to check if a specific test case is available, you can use the GET `testcases/{testcase_name}` API. If the test case is available, you will get a response with status code `200` that will confirm the test case is available. 

In [ ]:
testcase_names = ["multizone_office_simple_air", "example_testcase_not_present"]

for testcase_name in testcase_names:
  testcase_presence = requests.get(f"{BOPTEST_URL}/testcases/{testcase_name}")
 
  if testcase_presence.status_code == 200:
    print(f"Yes! \"{testcase_name}\" is present in list of testcases!")
  else:
    print(f"Failed to confirm \"{testcase_name}\" presence with response code returning {testcase_presence.status_code} :\\")

Yes! "multizone_office_simple_air" is present in list of testcases!
Failed to confirm "example_testcase_not_present" presence with response code returning 404 :\


Ok, there  are so many test cases available, how can I choose one to work with?

## Choosing a test case to work with: the POST `testcases/{testcase_name}/select` API 

The POST `testcases/{testcase_name}/select` API will help you choose a test case to work with. You get a `test_id` as a response when you call the `/select` API.  
The `test_id` is a unique identifier for each test. Most of the test specific BOPTEST API's require the `test_id` if you want to interact with the test. 


In [ ]:
testcase_name = "bestest_hydronic_heat_pump"

test_id = requests.post(f"{BOPTEST_URL}/testcases/{testcase_name}/select").json()['testid']
print(f"The test id for this particular test is {test_id}.")

We selected the "bestest_hydronic_heat_pump" test case and got a unique testid for interacting with the test. What all can I do with this test?

## Getting to know your test:

### Initializing using Scenarios:

Scenarios are pre-defined time periods and conditions for each test. Setting a scenario will initialize the test to start at the beginning of the pre-defined time period. The available scenarios for each BOPTEST test case is provided in the [BOPTEST documentation](https://ibpsa.github.io/project1-boptest/testcases/index.html). For example, the "bestest_hydronic_heat_pump" we are using in this tutorial has the following time periods and conditions available for scenario.  

Time periods:
1. peak_heat_day
2. typical_heat_day

Electricity prices:
1. constant
2. dynamic
3. highly_dynamic

#### Setting the scenario:

You can use the PUT `/scenario` API to set the scenario. In the next cell, we will set the time period to peak_heat_day and the electricity price to constant for our test.

In [ ]:
# Initialize using /scenario
y = requests.put(f'{BOPTEST_URL}/scenario/{test_id}', 
                 data={'time_period':'peak_heat_day',
                       'electricity_price':'constant'}).json() 

In [ ]:
print_BOPTEST_response(y)

Test case scenario was set successfully.
The payload is {
  "electricity_price": "constant",
  "time_period": {
    "ovePum_activate": 0,
    "weaSta_reaWeaNOpa_y": 0.2,
    "reaPFan_y": 510.20408163265313,
    "oveTSet_u": 294.34999999999997,
    "reaQHeaPumCon_y": 7386.771485507761,
    "reaTRet_y": 302.40029127477544,
    "weaSta_reaWeaPAtm_y": 101325,
    "weaSta_reaWeaTBlaSky_y": 260.82015225846817,
    "reaQHeaPumEva_y": -5001.493945256057,
    "weaSta_reaWeaNTot_y": 0.2,
    "weaSta_reaWeaSolAlt_y": -1.0200406146124426,
    "reaTZon_y": 294.4643919860063,
    "weaSta_reaWeaHHorIR_y": 262,
    "weaSta_reaWeaSolTim_y": 1379289.1064134557,
    "oveHeaPumY_u": 0.6283099681151593,
    "weaSta_reaWeaCloTim_y": 1382400,
    "oveHeaPumY_activate": 0,
    "reaPPumEmi_y": 20.498644281163177,
    "weaSta_reaWeaHGloHor_y": 0,
    "weaSta_reaWeaHDifHor_y": 0,
    "oveTSet_activate": 0,
    "weaSta_reaWeaRelHum_y": 0.9,
    "reaTSetHea_y": 294.15,
    "reaCO2RooAir_y": 754.1571682156247,
    

### Test measurements:

Measurements refer to the outputs from the test case model. Measurements are like properties that define the model state. Each test case has its specific set of measurements. You can check what measurements are available for a test with the GET `/measurements` request, as shown below. 

The API response will tell you what are 
1. The minimum values for the measurement.
2. A short description of the measurement, 
3. The measurement unit.
4. The maximum value for the measurement. 

 


In [ ]:
measurements_response = requests.get(f'{BOPTEST_URL}/measurements/{test_id}').json()
print_BOPTEST_response(measurements_response)

Queried the measurements successfully.
The payload is {
  "weaSta_reaWeaPAtm_y": {
    "Minimum": null,
    "Description": "Atmospheric pressure measurement",
    "Unit": "Pa",
    "Maximum": null
  },
  "reaPFan_y": {
    "Minimum": null,
    "Description": "Electrical power of the heat pump evaporator fan",
    "Unit": "W",
    "Maximum": null
  },
  "reaQHeaPumCon_y": {
    "Minimum": null,
    "Description": "Heat pump thermal power exchanged in the condenser",
    "Unit": "W",
    "Maximum": null
  },
  "reaTRet_y": {
    "Minimum": null,
    "Description": "Return water temperature from radiant floor",
    "Unit": "K",
    "Maximum": null
  },
  "weaSta_reaWeaNOpa_y": {
    "Minimum": null,
    "Description": "Opaque sky cover measurement",
    "Unit": "1",
    "Maximum": null
  },
  "weaSta_reaWeaTBlaSky_y": {
    "Minimum": null,
    "Description": "Black-body sky temperature measurement",
    "Unit": "K",
    "Maximum": null
  },
  "reaQHeaPumEva_y": {
    "Minimum": null,
   

### The control step:


This variable defines how much time the simulation will move forward at each step. 
There is a default value for the control step, so you don't have to set it if you don't want to. 


#### Getting the current/default control step: The GET `/step` API

This GET request will tell you what is the current control step is:

In [ ]:
# Get default control step
response = requests.get(f'{BOPTEST_URL}/step/{test_id}').json()
print(response['message'])

if response['status'] == 200:
  step = response['payload']
  print(f'The default step period is {step}s.')


Queried the control step successfully.
The default step period is 3600s.


#### Setting your own control step: The PUT `/step` API

There is a PUT `/step` API which allows you to set your own control step in seconds:

In [ ]:
# Set the control step
response = requests.put(f'{BOPTEST_URL}/step/{test_id}', data={'step':1800}).json()
print(response['message'])

if response['status'] == 200:
  step = response['payload']
  print(f'The control step period was set to {step}s.')


Control step set successfully.
The control step period was set to {'step': 1800}s.


In [ ]:
# Check the control step now with the GET /step API:
response = requests.get(f'{BOPTEST_URL}/step/{test_id}').json()
print(response['message'])

if response['status'] == 200:
  step = response['payload']
  print(f'The control step period now is {step}s.')

Queried the control step successfully.
The control step period now is 1800s.




---



## Moving forward with the simulation: The `/advance` API:

You step through the simulation using the `/advance` API. One call of the `/advance` API will move the simulation forward by one control step that we defined before. 

### Responses of the `/advance` API call:
The payload of the /advance call will show you the values of the current measurements. 

In [ ]:
# A basic advance step
y = requests.post(f'{BOPTEST_URL}/advance/{test_id}').json()

print_BOPTEST_response(y)

Advanced simulation successfully from 1382400.0s to 1384200.0s.
The payload is {
  "ovePum_activate": 0,
  "weaSta_reaWeaNOpa_y": 0.37,
  "reaPFan_y": 510.20408163265313,
  "oveTSet_u": 294.34999999999997,
  "reaQHeaPumCon_y": 6922.675155057209,
  "reaTRet_y": 302.3175489991955,
  "weaSta_reaWeaPAtm_y": 101325,
  "weaSta_reaWeaTBlaSky_y": 261.8763636716621,
  "reaQHeaPumEva_y": -4647.882674873364,
  "weaSta_reaWeaNTot_y": 0.37,
  "weaSta_reaWeaSolAlt_y": -1.0439994874090108,
  "reaTZon_y": 294.51991293043386,
  "weaSta_reaWeaHHorIR_y": 266.4642857142857,
  "weaSta_reaWeaSolTim_y": 1381088.6929458883,
  "oveHeaPumY_u": 0.5755813303128813,
  "weaSta_reaWeaCloTim_y": 1384200,
  "oveHeaPumY_activate": 0,
  "reaPPumEmi_y": 20.498644281163177,
  "weaSta_reaWeaHGloHor_y": 0,
  "weaSta_reaWeaHDifHor_y": 0,
  "oveTSet_activate": 0,
  "weaSta_reaWeaRelHum_y": 0.8944791666666667,
  "reaTSetHea_y": 294.15,
  "reaCO2RooAir_y": 766.39686084948,
  "weaSta_reaWeaSolDec_y": -0.3669202629349082,
  "oveP



---



## Interacting with your test:
## What are the inputs available for the testcase you selected? the GET `/inputs` API

The GET `/inputs` API will return what variables you can/have to set in the particular testcase you selected. 

The API response will tell you what are 
1. The minimum values for the variable.
2. A short description of the variable, 
3. The unit for the variable.
4. The maximum value for the variable. 

 

In [ ]:
# Get inputs available
inputs = requests.get(f'{BOPTEST_URL}/inputs/{test_id}').json()

print_BOPTEST_response(inputs)

Queried the inputs successfully.
The payload is {
  "oveTSet_activate": {
    "Minimum": null,
    "Description": "Activation for Zone operative temperature setpoint",
    "Unit": null,
    "Maximum": null
  },
  "ovePum_activate": {
    "Minimum": null,
    "Description": "Activation for Integer signal to control the emission circuit pump either on or off",
    "Unit": null,
    "Maximum": null
  },
  "ovePum_u": {
    "Minimum": 0,
    "Description": "Integer signal to control the emission circuit pump either on or off",
    "Unit": "1",
    "Maximum": 1
  },
  "oveHeaPumY_u": {
    "Minimum": 0,
    "Description": "Heat pump modulating signal for compressor speed between 0 (not working) and 1 (working at maximum capacity)",
    "Unit": "1",
    "Maximum": 1
  },
  "oveTSet_u": {
    "Minimum": 278.15,
    "Description": "Zone operative temperature setpoint",
    "Unit": "K",
    "Maximum": 308.15
  },
  "oveHeaPumY_activate": {
    "Minimum": null,
    "Description": "Activation for

### How to change the input values for the next control step:


The input variables can be set to desired values in a python dictionary.

In [ ]:
# Example input dictionary
u = {'oveHeaPumY_u':0.5,
     'oveHeaPumY_activate': 1}

### Advancing with the new input values

The input dictionary be sent as an /advance call body. This will cause the simulation to advance by one step with the new inputs. 

In [ ]:
# advance with u
y = requests.post(f'{BOPTEST_URL}/advance/{test_id}', data=u).json()

You will see the effects of this changed inputs in the response payload.

In [ ]:
print_BOPTEST_response(y)

Advanced simulation successfully from 1384200.0s to 1386000.0s.
The payload is {
  "ovePum_activate": 0,
  "weaSta_reaWeaNOpa_y": 0.6000000000000001,
  "reaPFan_y": 510.20408163265313,
  "oveTSet_u": 294.34999999999997,
  "reaQHeaPumCon_y": 6238.009336686093,
  "reaTRet_y": 302.06326258254757,
  "weaSta_reaWeaPAtm_y": 101325,
  "weaSta_reaWeaTBlaSky_y": 263.1259141058702,
  "reaQHeaPumEva_y": -4123.724172705812,
  "weaSta_reaWeaNTot_y": 0.6000000000000001,
  "weaSta_reaWeaSolAlt_y": -1.0485334341454244,
  "reaTZon_y": 294.5494328566635,
  "weaSta_reaWeaHHorIR_y": 272,
  "weaSta_reaWeaSolTim_y": 1382888.2797402868,
  "oveHeaPumY_u": 0.5,
  "weaSta_reaWeaCloTim_y": 1386000,
  "oveHeaPumY_activate": 1,
  "reaPPumEmi_y": 20.498644281163177,
  "weaSta_reaWeaHGloHor_y": 0,
  "weaSta_reaWeaHDifHor_y": 0,
  "oveTSet_activate": 0,
  "weaSta_reaWeaRelHum_y": 0.89,
  "reaTSetHea_y": 294.15,
  "reaCO2RooAir_y": 775.9299608657485,
  "weaSta_reaWeaSolDec_y": -0.36685410880578145,
  "ovePum_u": 1,
  


---

## Dynamically changing the inputs at every step: Make your own controller? 

A controller is an object that changes the input values for the next step based on the results from the last step. 

In the next cell you can see a basic controller implementation, which we have taken from <>.

In [ ]:
class Controller_Proportional(object):
    
    def __init__(self, TSet=273.15+21, k_p=10.):
        '''Constructor.

        Parameters
        ----------
        TSet : float, optional
            Temperature set-point in Kelvin.
        k_p : float, optional
            Proportional gain. 
            
        '''
        
        self.TSet = TSet
        self.k_p  = k_p
    
    def compute_control(self, y):
        '''Compute the control input from the measurement.
    
        Parameters
        ----------
        y : dict
            Contains the current values of the measurements.
            {<measurement_name>:<measurement_value>}
    
        Returns
        -------
        u : dict
            Defines the control input to be used for the next step.
            {<input_name> : <input_value>}
    
        '''
    
        # Compute control
        if y['reaTZon_y']<self.TSet:
            e = self.TSet - y['reaTZon_y']
        else:
            e = 0
    
        value = self.k_p*e
        u = {'oveHeaPumY_u':value,
             'oveHeaPumY_activate': 1}
    
        return u

## Using your own controller with BOPTEST:

In [ ]:
# Initialize scenario
y = requests.put(f'{BOPTEST_URL}/scenario/{test_id}', 
                 data={'time_period':'peak_heat_day',
                       'electricity_price':'dynamic'}).json()['payload']['time_period']

# Get the start time of the simulation
start_time_days = y['time']/24/3600
print(f"Start time of the simulation is {start_time_days} days from start of the year.")

# Set control step
requests.put(f'{BOPTEST_URL}/step/{test_id}', data={'step':3600})

# Instantiate controller
con = Controller_Proportional(TSet=273.15+21, k_p=5.)


Start time of the simulation is 16.0 days from start of the year.


In [ ]:
# Simulation loop
from IPython.display import clear_output
while y:
    # Clear the display output at each step
    clear_output(wait=True)
    # Print the current operative temperature and simulation time
    print('-------------------------------------------------------------------')
    print('Operative temperature [degC]  = {:.2f}'.format(y['reaTZon_y']-273.15))
    simulation_time_days = y['time']/3600/24
    print('Simulation time [elapsed days] = {:.2f}'.format((simulation_time_days - \
                                                    start_time_days)))
    print('-------------------------------------------------------------------')
    # Compute control signal 
    u = con.compute_control(y)
    # Advance simulation with control signal
    y = requests.post(f'{BOPTEST_URL}/advance/{test_id}', data=u).json()['payload']

-------------------------------------------------------------------
Operative temperature [degC]  = 20.80
Simulation time [elapsed days] = 14.00
-------------------------------------------------------------------


## How did your controller do? Looking at the KPIs:

KPIs(Key Performance Indicators) are a set of quantifiable measurements that will help you guage certain aspects of the controller performance. KPIs can be used to compare different controllers. For BOPTEST v0.3 we have the following KPIs: 

```
{
    "cost_tot":<value>,     // float, HVAC energy cost in $/m2 or Euro/m2
    "emis_tot":<value>,     // float, HVAC energy emissions in kgCO2e/m2
    "ener_tot":<value>,     // float, HVAC energy total in kWh/m2
    "pele_tot":<value>,     // float, HVAC peak electrical demand in kW/m2
    "pgas_tot":<value>,     // float, HVAC peak gas demand in kW/m2
    "pdih_tot":<value>,     // float, HVAC peak district heating demand in kW/m2
    "idis_tot":<value>,     // float, Indoor air quality discomfort in ppmh/zone
    "tdis_tot":<value>,     // float, Thermal discomfort in Kh/zone
    "time_rat":<value>      // float, Computational time ratio in s/ss
}
```

A GET `/kpi` request as shown below will tell you the KPIs calculated from the simulation start time till current simulation time. The warmup period is not taken into account for the KPI calculations. KPIs are usually calculated at the end of the simulation. 

In [ ]:
kpi_response = requests.get(f'{BOPTEST_URL}/kpi/{test_id}').json()
print_BOPTEST_response(kpi_response)

Queried KPIs successfully.
The payload is {
  "tdis_tot": 19.876021766419118,
  "idis_tot": 0,
  "ener_tot": 2.731379921546184,
  "cost_tot": 0.6932941788657855,
  "emis_tot": 0.4561404468982127,
  "pele_tot": 0.020371771206436497,
  "pgas_tot": null,
  "pdih_tot": null,
  "time_rat": 0.00011148731562315414
}


## Stopping the test:

The PUT `/stop` request will stop the testcase as shown below.



### Why it is important to stop tests? 
There are only a limited web-service resources available for running tests. If all the available resources are being used, the next incoming test will be queued up till the resources get freed up. So it's up to you to be responsible for your own tests. Take note of your test_id, and stop the test as soon as you are done using it.  

In [ ]:
stop_response = requests.put(f"{BOPTEST_URL}/stop/{test_id}")
if stop_response.status_code == 200:
  print("Successfully stopped the test!")
else:
  print(f"Couldn't stop test with status code: {stop_response.status_code}!")


Successfully stopped the test!
